## Packages and settings

In [36]:
import os, sys, pickle
import numpy as np 


import torch

from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.utils import get_density_thresholder, RestrictedPrior


In [37]:
from tqdm.notebook import tqdm
from nflows import distributions as distributions_
from nflows import flows, transforms
from nflows.nn import nets

In [38]:
# use GPU if available
device = 'cpu'
print('Device: %s'%(device))

Device: cpu


In [39]:
# settings for plots
import matplotlib
import matplotlib.pyplot as plt
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 10}
matplotlib.rc('font', **font)

rcnew = {"mathtext.fontset" : "cm", 
         "xtick.labelsize" : 18,
         "ytick.labelsize" : 18,
         "axes.titlesize" : 26, 
         "axes.labelsize" : 22,
         "xtick.major.size" : 8,      
         "xtick.minor.size" : 4,     
         "ytick.major.size" : 8,      
         "ytick.minor.size" : 4,      
         "legend.fontsize" : 22,
         'figure.titlesize' : 30,
         'errorbar.capsize' : 4,
         'axes.xmargin': 0.05,
          'axes.ymargin': 0.05,
        }
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')

%config InlineBackend.figure_format = 'retina'


In [40]:
num_dim = 2
low_priors  = torch.tensor([0.15, 0.65])
high_priors = torch.tensor([0.45, 0.95])
prior = utils.BoxUniform(low= low_priors, high=high_priors)

## Get the simulator

In [41]:
def build_maf(dim=1, num_transforms=8, context_features=None, hidden_features=128):
    transform = transforms.CompositeTransform(
        [
            transforms.CompositeTransform(
                [
                    transforms.MaskedAffineAutoregressiveTransform(
                        features=dim,
                        hidden_features=hidden_features,
                        context_features=context_features,
                        num_blocks=2,
                        use_residual_blocks=False,
                        random_mask=False,
                        activation=torch.tanh,
                        dropout_probability=0.0,
                        use_batch_norm=False,
                    ),
                    transforms.RandomPermutation(features=dim),
                ]
            )
            for _ in range(num_transforms)
        ]
    )

    distribution = distributions_.StandardNormal((dim,))
    neural_net = flows.Flow(transform, distribution)

    return neural_net


In [42]:
# load the emulator model
fmodel = ...

last_layer = 16
flow_net = build_maf(dim=last_layer, context_features=2).to(device=device)
flow_net.load_state_dict(torch.load(fmodel, map_location=torch.device(device)))
flow_net.eval(); 

In [43]:
def flow_simulator(theta, num_samples=1):
    """ 
    Simulator by sampling from the trained normalizing flow
    """
    x = flow_net.sample(num_samples=num_samples, context=theta).detach().numpy()
    return torch.tensor(x[:, 0, :])
    

In [44]:
# define observed summary vector s_obs; x_obs is the vector with true parameters
x_obs = torch.tensor([0.3, 0.8])
s_obs = torch.tensor([-0.0526,  0.0639,  0.1161,  0.1865,  0.2849, -0.2777,  
                      0.0077,  0.0468, -0.2226,  0.0172,  0.1345, -0.1732, -0.0653,  
                      0.1205, -0.1382, -0.1818])

## Load and sample the posterior obtained w/ emulator and sbi

In [ ]:
OmegaM, sigma8 = x_obs[0], x_obs[1]

posterior = pickle.load(open(.., "rb"))
posterior_samples = posterior.sample((10000,), x=s_obs)

# plot posterior samples
_ = analysis.pairplot(
    posterior_samples, limits=[[0.15, 0.45], [0.65, 0.95],], figsize=(5, 5)
)

## Plot posterior samples w/ getdist

In [48]:
# plot formatting 
import matplotlib
import matplotlib.pyplot as plt
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 10}
matplotlib.rc('font', **font)

rcnew = {"mathtext.fontset" : "cm", 
         "xtick.labelsize" : 18,
         "ytick.labelsize" : 18,
         "axes.titlesize" : 26, 
         "axes.labelsize" : 22,
         "xtick.major.size" : 8,      
         "xtick.minor.size" : 4,      
         "ytick.major.size" : 8,      
         "ytick.minor.size" : 4,      
         "legend.fontsize" : 22,
         'figure.titlesize' : 30,
         'errorbar.capsize' : 4,
         'axes.xmargin': 0.05,
          'axes.ymargin': 0.05,
        }
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')

In [ ]:
from getdist import plots, MCSamples
import getdist
names = ['OmegaM', 'sigma8']
labels =  ['\Omega_M', '\sigma_8']
params = x_obs.cpu().numpy()

posteriorSamples = MCSamples(samples=posterior_samples.cpu().numpy(), 
                             names = names, labels = labels, 
                             label='Emulator')

%matplotlib inline
g = plots.get_subplot_plotter(3.5)
g.settings.rc_sizes(legend_fontsize=18)
g.triangle_plot([posteriorSamples,],
                filled=False,
                markers={'OmegaM':params[0], 'sigma8':params[1]},
                labels = ['Emulator'],
                contour_colors=['forestgreen'], 
                contour_ls=[ '-'],
                contour_lws=2.5,
                legend_fontsize =40,
                legend_loc='upper right'
               )